# HTS [Part 6]

- Last Part!

In [1]:
%pylab inline
import sys
from ipywidgets import interact

Populating the interactive namespace from numpy and matplotlib


//anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
sys.path.append('src')
from motorControl import *
from trajectoryPlanning import *
from util import *

## Load Absolute Home, Upper right corner, out of the way, aligned with sharpie marks
f = open('savedPositions/absolute/home.p', 'r')
home = pickle.load(f)
f.close()

In [3]:
rc = connect(portName = "/dev/tty.usbserial-A9ETDN3N")
rc.Open()

1

### Initialized Motor Classes

- Really important to initialze these first to restore positions from last session!

In [4]:
#Initialize motor objects for each motor:
vL = Motor(address = 0x81, motorNumber = 2, rc = rc, signFlipped = True, \
           motorCounter = 0, kPID = [1e-2, 1.0])
vR = Motor(address = 0x81, motorNumber = 1, rc = rc, signFlipped = True, \
           motorCounter = 1, kPID = [1e-2, 1.0])
LR = Motor(address = 0x80, motorNumber = 1, rc = rc, signFlipped = True, 
           motorCounter = 2, kPID = [1e-2, 1.0])
FB = Motor(address = 0x80, motorNumber = 2, rc = rc, signFlipped = False, \
           motorCounter = 3, kPID = [1e-2, 1.0])
yaw = Motor(address = 0x82, motorNumber = 2, rc = rc, signFlipped = True, \
            motorCounter = 4, kPID = [1e-2, 1.0])
pitch = Motor(address = 0x82, motorNumber = 1, rc = rc, signFlipped = True, \
              motorCounter = 5, kPID = [1e-2, 1.0])

#Keep in a nice motor list:
motors = [vL, vR, LR, FB, yaw, pitch]

In [5]:
getPositions(motors)

[-66949, 66942, 50559, -85112, -97987, -62180]

In [6]:
home

[-1047, -2826, 0, 0, 0, -2]

In [21]:
MC = ManualControl(rc)
interact(MC.manualControl, leftUD = (-50, 50), rightUD = (-50, 50), leftRight = (-127, 127), \
         fB = (-127, 127), tilt = (-50, 50), pan = (-50, 50))

<function ipywidgets.widgets.interaction.<lambda>>

In [22]:
stopAll(rc)
savePositions(motors)
getPositions(motors)

[-806, -161, 50557, -134357, -131247, -43130]

In [12]:
stopAll(rc)
savePositions(motors)
getPositions(motors)

[-56440, 57491, -4306, -55533, -65488, -89208]

In [13]:
# levelRig(motors, home, rc, moveTime = 5.0)

## Save absolute home

- Only needed for hard resets

In [26]:
# #Upper right corner, out of the way, aligned with sharpie marks
# f = open('savedPositions/absolute/home.p', 'wb')
# pickle.dump(getPositions(motors), f, protocol=pickle.HIGHEST_PROTOCOL)
# f.close()

## Save Relative Position

In [70]:
# delta = np.array(getPositions(motors))-np.array(home)
# f = open('savedPositions/relative/HTS6/N.p', 'wb')
# pickle.dump(delta, f, protocol=pickle.HIGHEST_PROTOCOL)
# f.close()

## Load Relative Positions

In [71]:
#Position J
f = open('savedPositions/relative/HTS6/J.p', 'r')
J = pickle.load(f)+np.array(home) #Return to absolute coords
f.close()

#Position K
f = open('savedPositions/relative/HTS6/K.p', 'r')
K = pickle.load(f)+np.array(home) #Return to absolute coords
f.close()

#Position M
f = open('savedPositions/relative/HTS6/M.p', 'r')
M = pickle.load(f)+np.array(home) #Return to absolute coords
f.close()

#Position N
f = open('savedPositions/relative/HTS6/N.p', 'r')
N = pickle.load(f)+np.array(home) #Return to absolute coords
f.close()

In [15]:
J

array([-67006,  66986,  -4304,  -5138, -65488, -72221])

In [16]:
K

array([-56553,  57593,  -4304, -55436, -65488, -89172])

In [13]:
getPositions(motors)

[-56580, 57628, -4304, -55277, -65488, -89101]

In [17]:
L = getPositions(motors)
L[0] = L[0] + 25000
L[1] = L[1] - 25000

In [18]:
L

[-31440, 32491, -4306, -55533, -65488, -89208]

## Move the rig

In [116]:
targetPositions = N
totalTime = 20.0
rampTime = 4.0

startingPositions = getPositions(motors)
lookAheadTime = 1.0
tolerance = 100.0 #Anything less than this many ticks we're calling "Not a move"

motorsToMove = []
targetPositionsToMove = []
for i, motor in enumerate(motors):
    if abs(motor.getPosition()-targetPositions[i]) > tolerance:
        motorsToMove.append(motor)
        targetPositionsToMove.append(targetPositions[i])
        
trajectories = []
for i, motor in enumerate(motorsToMove):
    trajectories.append(SimpleQuadraticTrajectory(tu = rampTime, tt = totalTime, \
                                            p1 = motor.getPosition(), p2 = targetPositionsToMove[i]))
    
print np.array(targetPositions) - startingPositions
print 'Planning to move ' +  str(len(motorsToMove)) + ' motors.'

[     0      0      0 -60294      0  -8937]
Planning to move 2 motors.


In [117]:
for motor in motorsToMove:
    motor.initialize(targetVelocityMin = -25000.0, targetVelocityMax = 25000.0)
    motor.clearTracking()
    
startTime = time.time()
timeElapsed = 0.0
    
while timeElapsed < totalTime:
    timeElapsed = time.time()-startTime
    
    for i, motor in enumerate(motorsToMove):
        lookAheadValue = trajectories[i].compute(timeElapsed + lookAheadTime)
        motor.controlledMove(targetPosition = lookAheadValue, timeToReach = lookAheadTime)

stopAll(rc)
savePositions(motors)
print getPositions(motors), targetPositions
print getPositions(motors) - np.array(targetPositions)

[-66949, 66942, -4306, -48630, -65488, -68558] [-66949  66942  -4306 -49012 -65488 -68631]
[  0   0   0 382   0  73]


In [30]:
stopAll(rc)
savePositions(motors)

In [ ]:
A

In [ ]:
getPositions(motors)